In [3]:
######### GIADA BARZAGHI - AUGUST 2021
######### doing some practice to understand network modeling in Python 
######### starting point: P2P aviation network ---> goal: integrate itineraries

# SETTING UP WORKING ENVIRONMENT
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# per ottimizzazione 
from pyomo.environ import *

In [4]:
# SET DECLARATION 

# set of airports
N = [n for n in range(1,20)]
# set of aircraft models -------> import aircraft types 
A = ['B738','B773','A321','A346']
# set of flight legs  -----> NEED TO IMPLEMENT CONCEPT OF ITINRARIES!!!!
I = [(n1, n2) 
    for n1 in N 
    for n2 in N 
     if n1!=n2]
# have a look at the flight legs generated -> p2p there is a direct connection between each airport
print(I)
len(I) # 342 pairs generated when considering 20 airports p2p


[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 18), (4, 19), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (7

342

In [8]:
# 1.  Assigning a random capacity to each aircraft model 
np.random.seed(12345)
capacity = {}
for x in A:
    capacity[x]=np.random.randint(200, 500)
# print('capacity for each aircraft model: '+str(capacity))

# 2. Assigning a random distance to each flight leg
demand = {}
for i in I:
    demand[i]=100+np.random.rand()*1000
    #for key, value in distance.items():
        #print(key, value)
        
# 3. Assigning random values for pollution to each aircraft type when on a specific itinerary 
pollution = {}
for i in I:
    for a in A:
        pollution[i, a]=np.random.randint(50, 1000)

#for key, value in pollution.items():
#   print(key, value)

# 4. Creating a dummy for whether a flight-leg can be executed or not with a specific aircraft model:
# (0.5 probability of being yes/1)
rho = {}
for i in I:
    for a in A:
        if np.random.rand()<=0.5:
            rho[i, a]=1
        else:
            rho[i, a]=0
            
#print(rho)

# 5. Assigning random values for max utilization of each aircraft type
max_util = {}
for a in A:
    max_util[a] = np.random.randint(10, 50)

#for key, value in max_util.items():
#    print(key, value)


In [17]:
# MODEL SET UP 

# initializing parmaters and sets
model = ConcreteModel()
model.I = Set(initialize=I)
model.A = Set(initialize=A)
model.capacity=Param(model.A, initialize=capacity)
model.max_util = Param(model.A, initialize = max_util)
model.demand=Param(model.I, initialize=demand)
model.pollution=Param(model.I, model.A, initialize=pollution)
model.rho=Param(model.I, model.A, initialize=rho)

# setting up decision variables
model.frequency=Var(model.I, model.A, domain=NonNegativeIntegers)
model.z=Var(model.A, domain=NonNegativeIntegers) # z= number of aircrafts type A

# setting up model objective and constraints
def minimize_pollution(model):
    return sum(model.frequency[i,a]*model.pollution[i,a]
              for i in model.I for a in model.A)
model.objective=Objective(rule=minimize_pollution, sense=minimize)

def utilization_rule(model, a):
    return sum(model.frequency[i,a]
              for i in model.I)<=model.max_util[a]*model.z[a]
model.utilization= Constraint(model.A, rule=utilization_rule)

def demand_rule(model, i1, i2):
    return sum(model.frequency[i1, i2, a]*model.capacity[a]
              for a in model.A)>=model.demand[i1, i2]
model.demand_rule = Constraint(model.I, rule = demand_rule)

def operational_rule(model, i1, i2):
    return sum(model.frequency[i,a] 
              for I in model.I)<=model.rho[i,a]*model.frequency[i,a]
model.operational_rule = Constraint(model.I, rule=operational_rule)

In [19]:
# calling the solver -> using cbc

#%%time # what do we need this for?
opt = SolverFactory('cbc', executable=r'/usr/local/bin/cbc')
opt.options['mipgap']=0.20 # what is mipgap?
results = opt.solve(model, tee=True) # what is tee?

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Aug 12 2020 

command line - /usr/local/bin/cbc -mipgap 0.2 -printingOptions all -import /var/folders/wt/wvx51pxn2213wbhxthm8s1380000gn/T/tmpczi0_x2m.pyomo.lp -stat=1 -solve -solu /var/folders/wt/wvx51pxn2213wbhxthm8s1380000gn/T/tmpczi0_x2m.pyomo.soln (default strategy 1)
No match for mipgap - ? for list of commands
No match for 0.2 - ? for list of commands
Option for printingOptions changed from normal to all
Presolve 342 (-347) rows, 1367 (-6) columns and 1367 (-1716) elements
Statistics for presolved model
Original problem has 1372 integers (0 of which binary)
Presolved problem has 1367 integers (0 of which binary)
==== 0 zero objective 718 different
==== absolute objective values 718 different
==== for integers 0 zero objective 718 different
==== for integers absolute objective values 718 different
===== end objective counts


Problem has 342 rows, 1367 columns (1367 with objective) and 1367 elements
There are 1367 single

In [28]:
# PRINTING SOME RESULTS
# 1. number of aircrafts for each model type
for a in model.A:
    print('Aircraft type: '+str(a))
    print('Number of aircrafts: '+str(model.z[a].value))

# 2. frequancy of flights on itinerary i operated with aircraft type a
for i in model.I:
    for a in model.A:
        print('On flight leg %s there should be %s flights operated with aircraft type %s' 
             % (i, model.frequency[i,a].value, a))

Aircraft type: B738
Number of aircrafts: 18.0
Aircraft type: B773
Number of aircrafts: 8.0
Aircraft type: A321
Number of aircrafts: 6.0
Aircraft type: A346
Number of aircrafts: 10.0
On flight leg (1, 2) there should be 0.0 flights operated with aircraft type B738
On flight leg (1, 2) there should be 4.0 flights operated with aircraft type B773
On flight leg (1, 2) there should be 0.0 flights operated with aircraft type A321
On flight leg (1, 2) there should be 0.0 flights operated with aircraft type A346
On flight leg (1, 3) there should be 0.0 flights operated with aircraft type B738
On flight leg (1, 3) there should be 0.0 flights operated with aircraft type B773
On flight leg (1, 3) there should be 0.0 flights operated with aircraft type A321
On flight leg (1, 3) there should be 3.0 flights operated with aircraft type A346
On flight leg (1, 4) there should be 0.0 flights operated with aircraft type B738
On flight leg (1, 4) there should be 0.0 flights operated with aircraft type B77

In [ ]:
# THINK ABOUT SOME NICE GRAPHS YOU CAN MAKE 
# NICE TO GIVE A RPRESENTATION OF THE NETWORK 